In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, load_from_disk

In [2]:
# Hugging Face에서 발급받은 토큰 입력
HF_TOKEN = ""

# Hugging Face API 토큰 설정
os.environ["HF_TOKEN"] = HF_TOKEN

In [ ]:
# GPU 확인 및 설정
# GPU가 사용 가능한 경우 GPU를 사용하고, 그렇지 않으면 CPU를 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

# 로컬 경로 설정
MODEL_PATH = "./model"  # 모델이 다운로드된 경로
DATASET_PATH = "./dataset"  # 데이터셋이 다운로드된 경로

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device) # 모델을 GPU/CPU로 로드

# 데이터셋 로드
# dataset = load_dataset(DATASET_PATH)
dataset = load_from_disk(DATASET_PATH)

In [4]:
# 데이터셋 크기 감소 함수 정의
# 전체 데이터셋 크기를 줄여 학습 시간을 단축 (기본: 10%)
def sample_dataset(dataset, fraction=0.1):
    return dataset.shuffle(seed=42).select(range(int(len(dataset) * fraction)))

# 학습(train) 및 검증(validation) 데이터셋 샘플링
train_dataset = sample_dataset(dataset['train'], fraction=0.05)    # 학습 데이터의 5%만 사용
val_dataset = sample_dataset(dataset['validation'], fraction=0.05) # 검증 데이터의 5%만 사용

In [ ]:
# 데이터 전처리 함수 정의
# 모델 입력과 타겟(summary)을 토크나이저로 처리 (길이를 맞추기 위해 패딩과 트렁케이션 적용)
def preprocess_data(examples):
    inputs = examples['document'] # 본문
    targets = examples['summary'] # 요약문
    
    # 입력 텍스트를 모델에 맞게 토큰화, 최대 길이로 패딩 추가
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )

    # 타겟 텍스트도 동일하게 토큰화 및 패딩
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, max_length=128, truncation=True, padding="max_length"
        )
    
    # 입력 텍스트와 타겟 텍스트를 매핑
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터 전처리 함수 적용 (batch 단위로 처리)
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)

In [6]:
# 원본 데이터 컬럼(document와 summary) 제거
train_dataset = train_dataset.remove_columns(['document', 'summary'])
val_dataset = val_dataset.remove_columns(['document', 'summary'])

In [ ]:
# 학습 설정 정의
# 모델 학습에 필요한 다양한 설정을 포함
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",        # 학습 결과를 저장할 디렉토리
    evaluation_strategy="epoch",   # 매 에포크마다 검증 실행
    learning_rate=5e-5,            # 학습률 설정
    per_device_train_batch_size=8, # GPU 한 대당 학습 배치 크기
    per_device_eval_batch_size=8,  # GPU 한 대당 검증 배치 크기
    num_train_epochs=3,            # 학습 에포크 수
    save_strategy="epoch",         # 매 에포크마다 모델 저장
    logging_dir="./logs",          # 로그를 저장할 디렉토리
    logging_strategy="steps",      # 몇 스텝마다 로그 저장
    logging_steps=100,             # 100 스텝마다 로그 기록
    save_total_limit=2,            # 저장할 체크포인트 파일 수 제한
    predict_with_generate=True,    # 검증 시 요약 생성 활성화
    fp16=True,                     # Mixed Precision 학습 활성화 (FP16)
    push_to_hub=False              # Hugging Face Hub로 푸시 비활성화
)

# Trainer 초기화
# 모델, 데이터셋, 설정을 Trainer 객체로 전달
trainer = Seq2SeqTrainer(
    model=model,                   # 학습시킬 모델
    args=training_args,            # 학습 설정
    train_dataset=train_dataset,   # 학습 데이터셋
    eval_dataset=val_dataset,      # 검증 데이터셋
    tokenizer=tokenizer            # 토크나이저
)

In [ ]:
# 학습 실행
# Trainer 객체의 train() 메소드를 호출하여 학습 시작
trainer.train()

In [ ]:
# 학습 완료 후 모델 저장
trainer.save_model("./fine_tuned_model")        # Fine-tuned 모델 저장
tokenizer.save_pretrained("./fine_tuned_model") # 토크나이저 저장

print("Fine-tuned model saved to './fine_tuned_model'")